In [1]:
import os, sys, shutil, subprocess
from pathlib import Path

os.environ['PYTHONNOUSERSITE'] = '1'
os.environ.pop('PIP_TARGET', None)

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

# Clean entire PIP_TARGET and caches
for p in [
    "/app/.pip-target",
    str(Path("~/.pip-target").expanduser()),
    str(Path("~/.cache/pip").expanduser()),
    str(Path("~/.cache/huggingface").expanduser()),
    str(Path("~/.cache/torch").expanduser()),
]:
    shutil.rmtree(p, ignore_errors=True)

# Set writable PIP_TARGET
pip_target = Path('/app/.pip-target').resolve()
pip_target.mkdir(parents=True, exist_ok=True)
os.environ['PIP_TARGET'] = str(pip_target)
print(f'PIP_TARGET set to: {os.environ["PIP_TARGET"]}')

# 0) Hard reset any prior torch stacks
for pkg in ("torch","torchvision","torchaudio"):
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    "/app/.pip-target/torch",
    "/app/.pip-target/torch-2.8.0.dist-info",
    "/app/.pip-target/torch-2.4.1.dist-info",
    "/app/.pip-target/torch-2.5.1.dist-info",
    "/app/.pip-target/torchvision",
    "/app/.pip-target/torchvision-0.23.0.dist-info",
    "/app/.pip-target/torchvision-0.19.1.dist-info",
    "/app/.pip-target/torchvision-0.20.1.dist-info",
    "/app/.pip-target/torchaudio",
    "/app/.pip-target/torchaudio-2.8.0.dist-info",
    "/app/.pip-target/torchaudio-2.4.1.dist-info",
    "/app/.pip-target/torchaudio-2.5.1.dist-info",
    "/app/.pip-target/torchgen",
    "/app/.pip-target/functorch",
    "/app/.pip-target/transformers",
    "/app/.pip-target/transformers-4.44.2.dist-info",
    "/app/.pip-target/transformers-4.45.2.dist-info",
    "/app/.pip-target/accelerate",
    "/app/.pip-target/accelerate-0.34.2.dist-info",
):
    if os.path.exists(d):
        print("Removing", d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install the EXACT cu121 torch stack FIRST (matches your CUDA 12.1 container) - using 2.5.1 as per coach for compatibility
pip("install",
    "--index-url", "https://download.pytorch.org/whl/cu121",
    "--extra-index-url", "https://pypi.org/simple",
    "torch==2.5.1", "torchvision==0.20.1", "torchaudio==2.5.1")

# 2) Create a constraints file to freeze torch versions for all later installs
Path("constraints.txt").write_text(
    "torch==2.5.1\n"
    "torchvision==0.20.1\n"
    "torchaudio==2.5.1\n")

# 3) Now install NON-torch deps, honoring constraints, and avoid upgrading torch - using compatible versions
pip("install", "-c", "constraints.txt",
    "transformers==4.45.2", "accelerate==0.34.2",
    "datasets==2.21.0", "evaluate==0.4.2",
    "sentencepiece", "scikit-learn",
    "--upgrade-strategy", "only-if-needed")

# 4) Sanity gate (hard fail on drift) - but defer to next cell after restart
os._exit(0)

PIP_TARGET set to: /app/.pip-target


> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 368.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 302.8 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 487.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 297.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 276.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 KB 475.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 293.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 270.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 KB 474.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 522.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 KB 511.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 KB 510.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 KB 543.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 489.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 KB 511.4 MB/s eta 0:00:00
